In [1]:
%cd /Users/jye98/Desktop/dataset/imagedata

/Users/jye98/Desktop/dataset/imagedata


In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import os

In [ ]:
image_datas = glob.glob('./**/*.jpg')
class_name = ["StyleGAN3", "RealImage"]
dic = {"StyleGAN3":0, "RealImage":1}

In [ ]:
x = []
y = []
for imagename in image_datas:
    image = Image.open(imagename)
    image = image.resize((384,384))
    image = np.array(image)
    x.append(image)
    label = imagename.split(('/'),maxsplit=2)[1]
    label = dic[label]
    y.append(label)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(x, y, test_size=0.3, shuffle= True, random_state=777)
train_labels = train_labels[..., tf.newaxis]
test_labels = test_labels[..., tf.newaxis]

test_images.shape, test_labels.shape, train_images.shape, train_labels.shape

In [ ]:
#training set의 각 class 별 image 수 확인
unique, counts = np.unique(np.reshape(train_labels, (1160,)), axis=-1, return_counts=True)
dict(zip(unique, counts))

In [ ]:
N_TRAIN = train_images.shape[0]
N_TEST = test_images.shape[0] 

In [ ]:
train_images = train_images.astype(np.int32) / 255.
test_images = test_images.astype(np.int32) / 255.
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [ ]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

In [ ]:
plt.imshow(test_images[0])

In [ ]:
N_EPOCHS = 20
N_BATCH = 4
N_CLASS = 5

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(buffer_size=1243).batch(N_BATCH).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(N_BATCH)

In [ ]:
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.applications import DenseNet201
import tensorflow as tf
tf.keras.applications.DenseNet201(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000
)

import matplotlib.pyplot as plt
import math

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import binary_crossentropy
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

import matplotlib.pyplot as plt
import math

In [ ]:
densenet201_base = Xception(weights='imagenet',
                            include_top = False,
                            input_shape = (384,384,3))

model = models.Sequential()
model.add(densenet201_base)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

for layer in densenet201_base.layers:
    layer.trainable = False

for layer in densenet201_base.layers[15:]:
    layer.trainable = True
    
loss = binary_crossentropy
model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])

densenet201_base.summary()

In [ ]:
steps_per_epoch = math.ceil(N_TRAIN/N_BATCH)
validation_steps = math.ceil(N_TEST/N_BATCH)
print(steps_per_epoch, validation_steps)

In [ ]:
hist = model.fit(train_dataset, epochs=N_EPOCHS,steps_per_epoch=steps_per_epoch,validation_data=test_dataset,validation_steps=validation_steps,verbose=1)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
plt.plot(hist.history['loss'], '-b', label ='loss')
plt.plot(hist.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()


In [ ]:
plt.plot(hist.history['accuracy'], 'b-', label='accuracy')
plt.plot(hist.history['val_accuracy'], 'r--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()